In [ ]:
import numpy as np
import cv2
import scipy.optimize as opt
np.set_printoptions(precision=None, suppress=True)

In [ ]:
#number of matching points
N = 30
n_images = 8
F = np.zeros((n_images,n_images,3,3))
true_K = np.array([
    [5,0,2],
    [0,5,2],
    [0,0,1]
    ])
I = np.array([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0]
    ])

In [ ]:
def calc_fund_matrix(points1,points2):
    '''
    Takes in 2 arrays of matching points 
    Return Fundamental matrix computes 
    using 8 point algorithm
    '''
    F, mask = cv2.findFundamentalMat(points1,points2,cv2.FM_8POINT)
    return F 

In [ ]:
def normalize_coordinates(p):
    # Normalization matrix
    T_m = normalization_matrix(p)
    # Normalized points
    p_norm = (T_m@(p.T)).T
    return p_norm, T_m

    
def manual_find_F_matrix(xy1,xy2):
    # Input: xy1 , xy2 - 2d array of points: row per point [x,y]
    # xy1 - for image 1, xy2 - for image 2
    # FM compute fundametal matrix from point correspondence
    #    at least 8 points are needed
    
    n_points = xy1.shape[0]
    if n_points < 8:
        print('8 points at least are needed')
    
    #add 3rd coordinate '1'
    xy1 = projectivation(xy1)
    xy2 = projectivation(xy2)
    
    
    #normalize coordinates
    xy1_norm, T1 = normalize_coordinates(xy1)
    xy2_norm, T2 = normalize_coordinates(xy2)
    
    # Preparing the equation matrix
    M = np.zeros((n_points,9))
    M[:,0] = xy1_norm[:,0]*xy2_norm[:,0] # x1 * x2
    M[:,1] = xy1_norm[:,0]*xy2_norm[:,1] # x1 * y2 
    M[:,2] = xy1_norm[:,0] # x1
    M[:,3] = xy1_norm[:,1]*xy2_norm[:,0] # y1 * x2
    M[:,4] = xy1_norm[:,1]*xy2_norm[:,1] # y1 * y2
    M[:,5] = xy1_norm[:,1] # y1
    M[:,6] = xy2_norm[:,0] # x2
    M[:,7] = xy2_norm[:,1] # y2
    M[:,8] = 1

    # calc M^T * M to get symmetruc square matrix
    W = M.T@M
    
    # desompose W into U (singular values), D (diagonal matrix), Vt (singular vectors)
    U,D,Vt = la.svd(W,compute_uv=True)
    a = U[:,-1]     # solution vector corresponding to the 
                    # least singular value
    A = np.reshape(a.T,(3,3))

    # Returning to non-normalized points
    F = la.inv(T2)@A@T1

    return F

In [ ]:
### function used in class
def projectivation(p):
    '''
    Input: p, Nxd matrix = N points in R^d
    Output: q, Nx(d+1) = N points in P^d
    '''
    N,d = p.shape
    q = np.ones((N,d+1))
    q[:,0:d] = p 
    return q 

def affinization(q):
    '''
    Input: q, Nx(d+1) = N points in P^d
    Output: p, Nxd matrix = N points in R^d
    '''
    N,d1 = q.shape
    d = d1-1
    p = np.ones((N,d))
    p = q[:,0:d]/q[:,d:d+1] 
    
    return p

def normalization_matrix(p):
    '''
    Input: p: Nx2 matrix
    Output: T, normalization matrix (in projective plane) 
    '''

    # Computation
    m = np.mean(p,axis=0)
    q = p - np.repeat(m[np.newaxis,:],N,axis=0)
    w = np.sqrt(np.sum(q**2,axis=1))
    scale = 1/np.mean(w)

    # Normalization matrix
    T = np.zeros((3,3))
    T[2,2] = 1
    T[0,0] = scale
    T[0,2] = -m[0]*scale
    T[1,1] = scale
    T[1,2] = -m[1]*scale
        
    return T

In [ ]:
def mendonca_cost_func(X):
    '''
    computes Mendonca & Cipolla Cost function to find the Optimal Intrinsic Parameters
    Input
    X      - Approximate Values of Intrinsics - 1D array with length 5
    Output
    E    - Computed Cost
    '''

    #Transform Intrinsics to Matrix Form
    K = np.array([
        [X[0],X[1],X[2]],
        [0,X[3],X[4]],
        [0,0,1]
    ])
    #Initialize Cost
    E = 0
    '''
       F - globally defined Fundamental Matrix between given two Images (F[i,j,3x3])
    '''
    # For the Denominator term of Mendonca & Cipolla's Equation
    N = len(F[0]) #number of images given

    Den = N*(N-1)/2; # For N Images there are N(N-1)/2 Fundamental Matrices

    #Compute the Cost using Mendonca & Cipolla's Equation
    for i in range(1,len(F[0]-1)):
        for j in range (i+1,len(F[0])):

            # Compute the Essential Matrix 'EM' from Fundamental of images i,j
            EM = K.T @ F[i,j,:,:] @ K

            # Compute SVD of Essential Matrix
            _,D,_ = np.linalg.svd(EM)
             # Singular Values (3rd value, D[3] is 0 according to theorem)
            r = D[0]
            s = D[1]
            
            #  Compute Cost

            E+= (1/Den) * (r - s)/s

    return E


In [ ]:
# EXPERIMENT

# array of n_images with original coordinates of N points (x,y)
p = np.zeros((n_images,N,2))
# array of n_images with projective coordinats of M points (x,y,1)
pj = np.zeros((n_images,N,3))

# randomly initialize 1st image (index #0)
p[0] = np.random.randint(0,500,(N,2))
pj[0] = projectivation(p[0])

#init random matrices to transform original image
A = np.random.randint(-6,6,(n_images-1,3,3))
A[:,] = np.triu(A[:,])
# transform original image to get new images 
for i in range(1,n_images):
    pj[i] = (A[i-1]@pj[0].T).T
    p[i] = affinization(pj[i])
   
# insert noise
noise_level1 = 0.25
for i in range(n_images):
    p[i] += noise_level1*np.random.randn(N,2)

In [ ]:
print (A[0])

In [ ]:
print (p.shape)

In [ ]:
for i in range(n_images-1):
    for j in range(i+1,n_images):
        F[i,j] = calc_fund_matrix(p[i],p[j])

In [ ]:
initial_K =  np.random.randint(-5,5,(5))

In [ ]:
res = opt.minimize(mendonca_cost_func,x0=initial_K, method='Nelder-Mead')

In [ ]:
print (res)

In [ ]:
result_K = np.zeros((3,3))
result_K = np.array([[res.x[0],res.x[1],res.x[2]],[0,res.x[3],res.x[4]],[0,0,1]])
print (np.matrix(result_K))

In [ ]:
#reconststruction

#find Fundamental matrix
F12 = np.zeros((3,3))
F12 = calc_fund_matrix(p[0],p[1])

#find Essential matrix
EM = result_K.T @ F @ result_K

#factorize EM to find R,t
U,D,Vt = np.linalg.svd(EM)